# 라이브러리

In [45]:
import pandas as pd
import numpy as np
import os
import cv2


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [26]:
train_df = pd.read_csv("./open/train.csv")
train_df

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher
...,...,...,...
15829,./train/TRAIN_15829.jpg,./upscale_train/TRAIN_15829.png,Common Myna
15830,./train/TRAIN_15830.jpg,./upscale_train/TRAIN_15830.png,Common Kingfisher
15831,./train/TRAIN_15831.jpg,./upscale_train/TRAIN_15831.png,Cattle Egret
15832,./train/TRAIN_15832.jpg,./upscale_train/TRAIN_15832.png,Ruddy Shelduck


In [18]:
len(train_df['label'].unique())

25

In [38]:
test_df = pd.read_csv("./open/test.csv")
test_df

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg
...,...,...
6781,TEST_06781,./test/TEST_06781.jpg
6782,TEST_06782,./test/TEST_06782.jpg
6783,TEST_06783,./test/TEST_06783.jpg
6784,TEST_06784,./test/TEST_06784.jpg


In [24]:
pic_1 = cv2.imread('./open/upscale_train/TRAIN_00000.png')
pic_1.shape

(256, 256, 3)

In [31]:
low_pic_1 = cv2.imread('./open/train/TRAIN_00000.jpg')
print(low_pic_1.shape)

resize_img = np.array(cv2.resize(low_pic_1, (32,32)))
print(resize_img.shape)

(64, 64, 3)
(32, 32, 3)


# LeNet-5

* 초기의 CNN 구조
* 주로 32x32 크기의 저해상도 이미지 분류에 유용

## 데이터 처리

* 방법 1 : 이미지 크기를 64 or 128 이기 때문에 32로 변환
* 방법 2 : 64 or 128 크기의 이미지 처리할 수 있는 모델 수정

In [27]:
le_train = train_df.copy()
le = preprocessing.LabelEncoder()
le_train['en_label'] = le.fit_transform(train_df['label'])
le_train

,img_path,upscale_img_path,label,en_label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck,19
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail,9
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock,13
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher,3
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher,3
...,...,...,...,...
15829,./train/TRAIN_15829.jpg,./upscale_train/TRAIN_15829.png,Common Myna,4
15830,./train/TRAIN_15830.jpg,./upscale_train/TRAIN_15830.png,Common Kingfisher,3
15831,./train/TRAIN_15831.jpg,./upscale_train/TRAIN_15831.png,Cattle Egret,2
15832,./train/TRAIN_15832.jpg,./upscale_train/TRAIN_15832.png,Ruddy Shelduck,19


### 이미지 크기 32로 낮추기

### train 전처리

In [53]:
def low_image(x):
    resize_img = np.array(cv2.resize(x, (32,32)))
    # resize_img = resize_img.reshape(-1,32,32,3).astype('float32')/255
    img_flatten = resize_img.flatten()
    return img_flatten

low_image_path = './open/train/'
file_list = os.listdir(low_image_path)
# print(file_list)
images = []

for i in file_list:
    img_path = os.path.join(low_image_path + i)
    img = cv2.imread(img_path)
    re_img = low_image(img)
    images.append(re_img)

x_df = pd.DataFrame(images)
x_df['label'] = le_train['en_label']
x_df


,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
0,106,117,131,113,125,137,110,125,134,114,...,88,100,106,87,99,105,84,96,102,19
1,158,174,185,114,148,157,148,174,186,174,...,127,137,135,81,125,118,65,143,131,9
2,207,213,220,207,213,220,207,213,220,207,...,187,191,192,183,188,193,178,186,195,13
3,197,205,198,198,206,199,199,207,200,199,...,225,223,212,226,224,213,227,225,214,3
4,77,78,74,65,66,62,60,61,57,65,...,53,63,71,69,80,85,63,72,75,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15829,98,101,86,131,131,119,150,150,138,55,...,101,159,142,83,129,115,37,71,60,4
15830,182,190,197,193,202,206,198,206,207,197,...,114,140,154,114,139,150,109,135,143,3
15831,66,96,87,57,89,80,67,102,90,78,...,29,73,59,54,105,86,91,147,124,2
15832,175,176,167,178,179,170,177,178,169,173,...,154,170,152,153,169,151,155,171,153,19


### test 전처리

In [ ]:
def low_image(x):
    resize_img = np.array(cv2.resize(x, (32,32)))
    # resize_img = resize_img.reshape(-1,32,32,3).astype('float32')/255
    img_flatten = resize_img.flatten()
    return img_flatten

low_image_path = './open/test/'
file_list = os.listdir(low_image_path)
# print(file_list)
images = []

for i in file_list:
    img_path = os.path.join(low_image_path + i)
    img = cv2.imread(img_path)
    re_img = low_image(img)
    images.append(re_img)

y_df = pd.DataFrame(images)
y_df


,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.933333,0.890196,0.823529,0.933333,0.890196,0.823529,0.933333,0.890196,0.823529,0.933333,...,0.839216,0.929412,0.894118,0.839216,0.929412,0.894118,0.839216,0.925490,0.890196,0.835294
1,0.298039,0.607843,0.447059,0.301961,0.627451,0.454902,0.286275,0.650980,0.450980,0.278431,...,0.643137,0.611765,0.666667,0.662745,0.635294,0.678431,0.674510,0.635294,0.658824,0.650980
2,0.321569,0.552941,0.498039,0.325490,0.525490,0.478431,0.282353,0.435294,0.396078,0.243137,...,0.396078,0.282353,0.443137,0.384314,0.309804,0.474510,0.411765,0.349020,0.521569,0.454902
3,0.862745,0.796078,0.713726,0.866667,0.800000,0.717647,0.870588,0.803922,0.721569,0.870588,...,0.619608,0.290196,0.470588,0.572549,0.384314,0.568627,0.686275,0.305882,0.486275,0.615686
4,0.784314,0.749020,0.694118,0.788235,0.752941,0.698039,0.788235,0.752941,0.698039,0.792157,...,0.576471,0.498039,0.568627,0.698039,0.376471,0.454902,0.600000,0.329412,0.415686,0.568627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6781,0.572549,0.572549,0.450980,0.725490,0.721569,0.607843,0.564706,0.611765,0.509804,0.517647,...,0.776471,0.333333,0.400000,0.745098,0.356863,0.482353,0.776471,0.203922,0.368627,0.635294
6782,0.490196,0.509804,0.513726,0.474510,0.498039,0.494118,0.474510,0.494118,0.490196,0.462745,...,0.333333,0.278431,0.333333,0.301961,0.278431,0.333333,0.317647,0.294118,0.349020,0.341176
6783,0.215686,0.231373,0.184314,0.211765,0.227451,0.180392,0.164706,0.180392,0.133333,0.160784,...,0.556863,0.262745,0.682353,0.576471,0.325490,0.717647,0.603922,0.325490,0.717647,0.592157
6784,0.301961,0.341176,0.407843,0.298039,0.329412,0.392157,0.305882,0.333333,0.388235,0.411765,...,0.388235,0.274510,0.309804,0.364706,0.235294,0.270588,0.317647,0.203922,0.227451,0.266667


### 데이터 분할

In [50]:
x_data = x_df.iloc[:, :-1].values
x_label = x_df['label'].values

x_train, x_val, x_train_label, x_val_label \
    = train_test_split(x_data, x_label, test_size = 0.2, random_state=1)

print(x_train.shape)
print(x_train_label.shape)
print(x_val.shape)
print(x_val_label.shape)

(12667, 3072)
(12667,)
(3167, 3072)
(3167,)


## 모델 생성

### 기본 모델

In [47]:
def create_lenet5(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(16, (5, 5), activation='relu', padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(120, (5, 5), activation='relu', padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(84, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model


input_shape = (32, 32, 3)
model_1 = create_lenet5(input_shape)

model_1.summary()

c:\Users\ehdwn\OneDrive\바탕 화면\project\code\bird_image\bird\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 6)      │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 16, 16, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 8, 8, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7680)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 84)             │       645,204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 697,046 (2.66 MB)

 Trainable params: 697,046 (2.66 MB)

 Non-trainable params: 0 (0.00 B)

#### 모델 학습

In [51]:
model_1.compile(optimizer='adam',
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
history_1 = model_1.fit(x_train, x_train_label, epochs=10, 
                        batch_size=64, validation_data=(x_val, x_val_label))

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 3072), dtype=float32). Expected shape (None, 32, 32, 3), but input has incompatible shape (None, 3072)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3072), dtype=float32)
  • training=True
  • mask=None